In [113]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences

要predict的是each rally result

In [198]:
game_num = 2
path = 'set1_test.csv'
df_combined = pd.DataFrame()
for i in range(0, game_num):
    df = pd.read_csv(path, dtype={'No.': str})
    df.insert(0, 'Game', i)
    df.insert(8, 'Nothing', np.nan)
    df = df.drop(index=df[df['Action'] == 'OP'].index.tolist())
    for idx, row in df.iterrows():
        if(row['Attempts'] == 1):
            df.loc[idx, 'Action'] = df.loc[idx, 'Action'] + '_1'
        else:
            df.loc[idx, 'Action'] = df.loc[idx, 'Action'] + '_0'
        if(pd.isna(row['Errors']) and pd.isna(row['Score'])):
            df.loc[idx, 'Nothing'] = 1
    
    df = df.drop(columns='Attempts')
    df_combined = pd.concat([df_combined, df])

df_combined = df_combined.reset_index(drop=True)
df_combined = df_combined.replace({'IRI': 0, 'POL': 1})
df_dummy = pd.get_dummies(df_combined, columns=['No.'])

In [199]:
space_replace = {list(df_dummy.groupby('Space').groups.keys())[i]: i+1 for i in range(len(df_dummy.groupby('Space')))}
action_replace = {list(df_dummy.groupby('Action').groups.keys())[i]: i+1 for i in range(len(df_dummy.groupby('Action')))}

In [200]:
space_replace

{'1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 'FZ': 10,
 'JS': 11,
 'L': 12,
 'M': 13,
 'Y': 14}

In [201]:
df_dummy = df_dummy.replace(space_replace)
df_dummy = df_dummy.replace(action_replace)

print(len(df_dummy))
df_dummy.head()

480


,Game,Team,Rally,Space,Action,Errors,Nothing,Score,No._,No._1,...,No._17,No._18,No._19,No._2,No._20,No._22,No._23,No._5,No._6,No._8
0,0,0,1,14,8,NaN,1.0,NaN,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,1,1,8,9,NaN,1.0,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,2,6,NaN,1.0,NaN,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,1,1,3,1,1.0,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,1,1,2,NaN,NaN,1.0,0,0,...,0,0,0,0,0,1,0,0,0,0


By game_dataframe 下去做可能會比較好一點

In [227]:
def process(df):
    '''
    one df represents one game
    split: space, action, player(?), error&score
    label: 發球方是否得分？
    '''
    # space_col = [c for c in df.columns if 'Space' in c]
    # action_col = [c for c in df.columns if 'Action' in c]
    result_col = ['Errors', 'Score', 'Nothing']
    others_col = [c for c in df.columns if c not in ['Space', 'Action', 'Game', 'Rally'] and c not in result_col]

    rally_set = []
    rally_space_set = []
    rally_action_set = []
    rally_result_set = []
    for _, df_rally in df.groupby(['Game', 'Rally']):   # each rally in one game
        curr_team = df_combined['Team'][df_rally.index[0]]
        shot_set = []
        shot_space_set = []
        shot_action_set = []
        shot_result_set = []
        
        atk_sequence = []
        atk_space_sequence = []
        atk_action_sequence = []
        atk_result = []
        
        for _, shot in df_rally.iterrows():
            if(shot['Team'] != curr_team):
                shot_set.append(atk_sequence)
                shot_space_set.append(atk_space_sequence)
                shot_action_set.append(atk_action_sequence)
                shot_result_set.append(atk_result)   # 最後一動的結果 -> predict object
                
                curr_team = shot['Team']

                atk_sequence = []
                atk_space_sequence = []
                atk_action_sequence = []

            atk_space_sequence.append([shot['Space'].copy().astype(float)])
            atk_action_sequence.append([shot['Action'].copy().astype(float)])
            atk_sequence.append(shot[others_col].copy().astype(float))
            atk_result = [shot[result_col].copy().astype(float)]
        
        # the last shot
        shot_set.append(atk_sequence)
        shot_space_set.append(atk_space_sequence)
        shot_action_set.append(atk_action_sequence)
        shot_result_set.append(atk_result)

        # one rally has been finished
        shot_set = pad_sequences(shot_set, maxlen=3, padding='post')
        shot_space_set = pad_sequences(shot_space_set, maxlen=3, padding='post')
        shot_action_set = pad_sequences(shot_action_set, maxlen=3, padding='post')

        rally_set.append(shot_set)
        rally_space_set.append(shot_space_set)
        rally_action_set.append(shot_action_set)
        rally_result_set.append(shot_result_set)

    padded_rally_set = pad_sequences(rally_set, padding='post')
    padded_rally_space_set = pad_sequences(rally_space_set, padding='post')
    padded_rally_action_set = pad_sequences(rally_action_set, padding='post')
    padded_rally_result_set = pad_sequences(rally_result_set, padding='post')
    return padded_rally_set, padded_rally_space_set, padded_rally_action_set, padded_rally_result_set

In [228]:
rally_set, rally_space_set, rally_action_set, rally_result_set = process(df_dummy)

# rally數, 最大回合數in one rally, 3, feature數
print(rally_set.shape)
print(rally_space_set.shape)
print(rally_action_set.shape)
print(rally_result_set.shape)

(86, 6, 3, 16)
(86, 6, 3, 1)
(86, 6, 3, 1)
(86, 6, 1, 3)


---

In [303]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.layers import Embedding, Masking, Concatenate, GRU, Dense, Reshape
import keras
from model import CNN_with_mask

In [250]:
rally_num = rally_set.shape[0]
rally_size = rally_set.shape[1]
shot_size = 3
feature_dim = (rally_set.shape[-1], rally_space_set.shape[-1], rally_action_set.shape[-1], rally_result_set.shape[-1])
space_embed_size = 8
action_embed_size = 8
shot_embed_size = 16

In [315]:
def create_model(feature_dim, space_embed_size, action_embed_size, shot_embed_size):
    '''
    framework: 
    1. 對 space, action 做 embeding, (input, output) = (feature_dim, embed_size)
    2. concat space, action, others 成一個 embedded vector for each atk, (input) =  ([feature_dim, embed_size, embed_size])
    3. 先過一個 GRU 變成一個 shot embedded vector, (input, output) = (concat_size, shot_embed_size)
    4. CNN, filters = shot_embed_size
    5. GRU
    '''
    # input transform
    input_others = keras.Input(shape=(rally_size, shot_size, feature_dim[0]))
    input_space = keras.Input(shape=(rally_size, shot_size, feature_dim[1]))
    input_action = keras.Input(shape=(rally_size, shot_size, feature_dim[2]))

    embed_space_layer = Embedding(input_dim=feature_dim[1], output_dim=space_embed_size, mask_zero=True, name='Space_Embedding')
    embed_action_layer = Embedding(input_dim=feature_dim[2], output_dim=action_embed_size, mask_zero=True, name='Action_Embedding')

    masking_layer = Masking(mask_value=0)   # for input_others (還沒有經過mask)

    concat_layer = Concatenate(name='Input_Concat')

    # use GRU to get shot embedding
    reshpae_layer = Reshape((18, 32))
    gru_layer = GRU(shot_embed_size, name='Shot_Embedding')
    # use CNN

    # use GRU

    dense_layer = Dense(units=3, activation='softmax')

    # forward
    inputs = [input_others, input_space, input_action]

    embed_space = tf.squeeze(embed_space_layer(input_space), axis=3)
    embed_action = tf.squeeze(embed_action_layer(input_action), axis=3)
    masked_others = masking_layer(tf.cast(input_others, tf.float32))
    embed_input = concat_layer([masked_others, embed_space, embed_action])

    # squeeze_embed_input = reshpae_layer(embed_input)
    # embed_shot = gru_layer(squeeze_embed_input)

    output_prob = dense_layer(embed_input)
    model = keras.Model(inputs=inputs, outputs=output_prob, name='Classification')
    return model

In [316]:
model = create_model(feature_dim, space_embed_size, action_embed_size, shot_embed_size)
model.summary()

Model: "Classification"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_140 (InputLayer)          [(None, 6, 3, 16)]   0                                            
__________________________________________________________________________________________________
input_141 (InputLayer)          [(None, 6, 3, 1)]    0                                            
__________________________________________________________________________________________________
input_142 (InputLayer)          [(None, 6, 3, 1)]    0                                            
__________________________________________________________________________________________________
tf.cast_38 (TFOpLambda)         (None, 6, 3, 16)     0           input_140[0][0]                  
_____________________________________________________________________________________